# 불러오기, df1 만들기

txt파일을 열어 확인 -> 'Time','CAN', 'Identifier', 'Flage', 'DLC', 'Data', 'Counter'
총 7가지 features가 있다.

txt형식의 데이터를 pandas.DataFrame 객체로 만들 수 있는 함수를 만들어 보자.
1) 파일을 로드해 line별로 읽는다
2) = 문자가 연속된 line의 인덱스를 찾는다
3) 그 다음 다음 line부터 순회하며 데이터를 적절한 리스트에 저장

In [1]:
cd /home/chlee/codes/automobile

/home/chlee/codes/automobile


In [2]:
def read_txt(file_path):
    # 리스트 여러개 리턴보다 딕셔너리로 묶어서 한번에 리턴이 나아보임.
    columns = {
        'Time': [],
        'CAN': [],
        'Identifier': [],
        'Flags': [],
        'DLC': [],
        'Data': [],  # 전체 데이터 문자열
        'Counter': [],
        'Error': []  # 오류 여부를 저장하는 새 컬럼 추가
    }
    
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        
        # 플래그 변수들
        found_separator = False
        skip_next_line = False
        
        for i, line in enumerate(lines):
            # '=' 구분자를 찾았는지 확인 (30개 이상)
            if '=' * 60 in line:
                found_separator = True
                skip_next_line = True
                continue
            
            # 구분자 다음 줄은 건너뛰기 (Trigger 정보 등)
            if skip_next_line:
                skip_next_line = False
                continue
                
            # 구분자를 찾았고, 그 다음다음 줄부터 데이터 처리
            if found_separator and not skip_next_line:
                # 공백으로 분리하여 데이터 추출
                parts = line.strip().split()
                
                if len(parts) < 4:  # 최소한 Time, CAN, Identifier, Flags는 있어야 함
                    print(f"Line {i+1}: 데이터 부족 - {line.strip()}")
                    continue
                
                try:
                    # 기본 컬럼 추출
                    time_val = parts[0]
                    can_channel = parts[1]
                    identifier = parts[2]
                    flags = parts[3]
                    
                    # ErrorFrame 특별 처리
                    if "ErrorFrame" in identifier:
                        # ErrorFrame이면 항상 "ERROR FRAME"이 있고, 마지막 필드가 Counter
                        # DLC 필드는 사용하지 않음
                        dlc = -1  # ErrorFrame의 경우 특수 값으로 표시
                        data_str = "ERROR FRAME"
                        counter = parts[-1]
                        is_error = True
                    else:
                        # 일반 프레임 처리
                        dlc = int(parts[4])  # DLC를 정수로 변환
                        # DLC 값에 따라 데이터 바이트 추출
                        data_bytes = parts[5:5+dlc]
                        data_str = " ".join(data_bytes)
                        # 남은 부분은 Counter
                        counter = parts[5+dlc] if 5+dlc < len(parts) else ""
                        is_error = False
                    
                    # 딕셔너리에 저장
                    columns['Time'].append(time_val)
                    columns['CAN'].append(can_channel)
                    columns['Identifier'].append(identifier)
                    columns['Flags'].append(flags)
                    columns['DLC'].append(dlc)
                    columns['Data'].append(data_str)
                    columns['Counter'].append(counter)
                    columns['Error'].append(is_error)
                    
                except (ValueError, IndexError) as e:
                    # 파싱 오류 발생 시 처리
                    print(f"Line {i+1} 데이터 파싱 오류: {e}")
                    print(f"오류 발생 line: {line.strip()}")
                    
                    # ErrorFrame 특별 처리를 다시 시도
                    if "ErrorFrame" in line and "ERROR" in line and "FRAME" in line:
                        # ErrorFrame 형식에 맞게 처리
                        error_parts = line.strip().split()
                        columns['Time'].append(error_parts[0] if len(error_parts) > 0 else "")
                        columns['CAN'].append(error_parts[1] if len(error_parts) > 1 else "")
                        columns['Identifier'].append("ErrorFrame")
                        columns['Flags'].append(error_parts[3] if len(error_parts) > 3 else "")
                        columns['DLC'].append(-1)
                        columns['Data'].append("ERROR FRAME")
                        columns['Counter'].append(error_parts[-1] if len(error_parts) > 5 else "")
                        columns['Error'].append(True)
                    else:
                        # 일반 오류 처리
                        columns['Time'].append(parts[0] if len(parts) > 0 else "")
                        columns['CAN'].append(parts[1] if len(parts) > 1 else "")
                        columns['Identifier'].append(parts[2] if len(parts) > 2 else "")
                        columns['Flags'].append(parts[3] if len(parts) > 3 else "")
                        columns['DLC'].append(-2)  # 기타 오류를 위한 또 다른 특수값
                        columns['Data'].append("")
                        columns['Counter'].append(parts[-1] if len(parts) > 4 else "")
                        columns['Error'].append(True)
    
    return columns

In [3]:
if __name__ == "__main__":
    file_path1 = ('./datasets/logfile2022-11-01_11-56-50.txt')
    file_path2 = ('./datasets/logfile2022-11-01_12-21-04.txt')
    file_path3 = ('./datasets/logfile2022-11-01_13-00-02.txt')
    file_path4 = ('./datasets/logfile2022-11-01_13-25-08.txt')
    file_path5 = ('./datasets/logfile2022-11-01_14-02-06.txt')
    file_path6 = ('./datasets/logfile2022-11-01_14-24-48.txt')
    data1 = read_txt(file_path1)
    # data2 = read_txt(file_path2)
    # data3 = read_txt(file_path3)
    # data4 = read_txt(file_path4)
    # data5 = read_txt(file_path5)
    # data6 = read_txt(file_path6)
    
    # 데이터 확인
    print(f"총 {len(data1['Time'])}개의 로그 항목을 읽었습니다.")
    print(f"정상 항목: {len(data1['Time']) - sum(data1['Error'])}개")
    print(f"오류 항목: {sum(data1['Error'])}개")
    
    # ErrorFrame 예시 출력 (data1 객체)
    error_indices = [i for i, err in enumerate(data1['Error']) if err]
    if error_indices:
        print("\nErrorFrame 예시:")
        idx = error_indices[0]
        print(f"Time: {data1['Time'][idx]}")
        print(f"CAN: {data1['CAN'][idx]}")
        print(f"Identifier: {data1['Identifier'][idx]}")
        print(f"Flags: {data1['Flags'][idx]}")
        print(f"DLC: {data1['DLC'][idx]}")
        print(f"Data: {data1['Data'][idx]}")
        print(f"Counter: {data1['Counter'][idx]}")

총 3123797개의 로그 항목을 읽었습니다.
정상 항목: 3123785개
오류 항목: 12개

ErrorFrame 예시:
Time: 6.862401
CAN: 1
Identifier: ErrorFrame
Flags: Rx
DLC: -1
Data: ERROR FRAME
Counter: 14923


In [4]:
type(data1)

dict

In [5]:
import pandas as pd
import numpy as np

def convert_to_dataframe(data_dict):
    """
    딕셔너리를 판다스 DataFrame으로 변환하는 함수
   
    Parameters:
    data_dict (dict): read_txt 함수로 얻은 딕셔너리
   
    Returns:
    pandas.DataFrame: 변환된 데이터프레임
    """
    # 입력 데이터 유효성 검사
    if not isinstance(data_dict, dict):
        raise TypeError(f"입력 데이터가 딕셔너리가 아닙니다. 현재 타입: {type(data_dict)}")
   
    # 빈 딕셔너리 처리
    if not data_dict:
        print("경고: 빈 딕셔너리가 입력되었습니다.")
        return pd.DataFrame()
    
    # Data 컬럼을 16진수로 변환 (존재하는 경우)
    if 'Data' in data_dict:
        # 'Data' 컬럼의 각 항목을 16진수로 변환
        try:
            new_data = []
            for data_item in data_dict['Data']:
                if pd.notna(data_item) and data_item != '':
                    # 'ERROR' 문자열 처리
                    if data_item == 'ERROR FRAME':
                        new_data.append('ERROR FRAME')
                        continue
                        
                    try:
                        # 공백으로 구분된 문자열을 숫자 리스트로 변환
                        nums = str(data_item).split()
                        
                        # 숫자들을 바이트 배열로 변환
                        byte_array = bytes([int(num) for num in nums])
                        
                        # 바이트 배열을 바이트 리터럴 형태로 저장
                        new_data.append(byte_array)
                    except ValueError:
                        print(f"경고: 숫자로 변환할 수 없는 Data 값 발견: {data_item}")
                        new_data.append(str(data_item))
                else:
                    new_data.append(b'')
            data_dict['Data'] = new_data
        except Exception as e:
            print(f"Data 컬럼 바이트 변환 중 오류 발생: {e}")
   
    # 1. 각 키의 값 길이가 동일한지 확인
    try:
        lengths = {}
        for key, value in data_dict.items():
            try:
                lengths[key] = len(value)
            except TypeError:
                print(f"경고: '{key}' 키의 값이 길이를 측정할 수 없는 타입입니다: {type(value)}")
                lengths[key] = 1  # 기본값 설정
       
        length_values = list(lengths.values())
    except Exception as e:
        print(f"길이 검사 중 오류 발생: {e}")
        return pd.DataFrame()  # 오류 발생 시 빈 DataFrame 반환
   
    if len(set(length_values)) != 1:
        # 길이가 일치하지 않는 경우
        print(f"경고: 딕셔너리의 값 길이가 일치하지 않습니다.")
        print("각 키별 길이:")
        for key, length in lengths.items():
            print(f"  - {key}: {length}")
       
        # 가장 짧은 길이로 자르기
        min_length = min(length_values)
        print(f"모든 값을 가장 짧은 길이({min_length})로 자릅니다.")
       
        # 각 값을 자르기
        for key in data_dict.keys():
            try:
                data_dict[key] = data_dict[key][:min_length]
            except TypeError:
                print(f"경고: '{key}' 키의 값은 슬라이스할 수 없습니다.")
   
    # 2. 데이터프레임으로 변환
    try:
        df = pd.DataFrame(data_dict)
    except Exception as e:
        print(f"DataFrame 생성 중 오류 발생: {e}")
        return pd.DataFrame()
   
    # 3. 데이터 타입 변환
    try:
        # Time을 float로 변환
        df['Time'] = pd.to_numeric(df['Time'], errors='coerce')
       
        # DLC를 정수로 변환 (ErrorFrame의 경우 -1로 이미 설정됨)
        df['DLC'] = pd.to_numeric(df['DLC'], errors='coerce').astype('Int64')  # nullable integer
       
        # Counter를 정수로 변환 (빈 문자열은 NaN으로)
        if 'Counter' in df.columns:
            df['Counter'] = pd.to_numeric(df['Counter'], errors='coerce').astype('Int64')
       
        # Error를 bool로 변환
        if 'Error' in df.columns:
            df['Error'] = df['Error'].astype(bool)
    except Exception as e:
        print(f"데이터 타입 변환 중 오류 발생: {e}")
   
    # 5. 결과 요약 정보 출력
    print(f"변환 완료: {len(df)}행, {len(df.columns)}열")
   
    try:
        if 'Error' in df.columns:
            error_count = df['Error'].sum()
            print(f"ErrorFrame 수: {error_count}개")
    except Exception as e:
        print(f"ErrorFrame 개수 계산 중 오류 발생: {e}")
   
    return df

In [6]:
# file_paths = [
#     ('./datasets/logfile2022-11-01_11-56-50.txt'),
#     ('./datasets/logfile2022-11-01_12-21-04.txt'),
#     ('./datasets/logfile2022-11-01_13-00-02.txt'),
#     ('./datasets/logfile2022-11-01_13-25-08.txt'),
#     ('./datasets/logfile2022-11-01_14-02-06.txt'),
#     ('./datasets/logfile2022-11-01_14-24-48.txt')
# ]

# 모든 파일 통합 처리
def raw_to_df(path):
    print('=' * 20, "raw data를 dataframe으로 변환 시도 중", '=' * 20)

    dict = read_txt(path)
    df = convert_to_dataframe(dict)

    return df

df1 = raw_to_df('./datasets/logfile2022-11-01_11-56-50.txt')
# df2 = raw_to_df('./datasets/logfile2022-11-01_12-21-04.txt')
# df3 = raw_to_df('./datasets/logfile2022-11-01_13-00-02.txt')
# df4 = raw_to_df('./datasets/logfile2022-11-01_13-25-08.txt')
# df5 = raw_to_df('./datasets/logfile2022-11-01_14-02-06.txt')
# df6 = raw_to_df('./datasets/logfile2022-11-01_14-24-48.txt')

==================== raw data를 dataframe으로 변환 시도 중 ====================
변환 완료: 3123797행, 8열
ErrorFrame 수: 12개


In [7]:
df1.head()

,Time,CAN,Identifier,Flags,DLC,Data,Counter,Error
0,0.026248,1,902,Rx,8,b'n\x84e\x04m\xc4c\xc4',1,False
1,0.026416,1,356,Rx,4,b'\x00\x08\x0c\x04',2,False
2,0.028296,1,1427,Rx,6,"b'\x00\x01!""""!'",3,False
3,0.028528,1,899,Rx,8,b'\x05(\x94qq\x00\x00\xb9',4,False
4,0.029188,1,897,Rx,8,b'\x80\x00@\x80\xffE\x85\x01',5,False


In [8]:
# df1에서 Error 컬럼의 값이 True인 행만 출력
(df1[df1["Error"]==True])

# 근데 생각해보니 python에서는 df['Error']가 이미 불리언 시리즈이므로 ==True는 생략 가능
#error_rows = df[df['Error']]

,Time,CAN,Identifier,Flags,DLC,Data,Counter,Error
14922,6.862401,1,ErrorFrame,Rx,-1,ERROR FRAME,14923,True
2035503,932.707966,1,ErrorFrame,Rx,-1,ERROR FRAME,2035504,True
3123787,1431.490367,1,ErrorFrame,Rx,-1,ERROR FRAME,3123788,True
3123788,1431.490401,1,ErrorFrame,Rx,-1,ERROR FRAME,3123789,True
3123789,1431.490427,1,ErrorFrame,Rx,-1,ERROR FRAME,3123790,True
3123790,1431.490445,1,ErrorFrame,Rx,-1,ERROR FRAME,3123791,True
3123791,1431.490461,1,ErrorFrame,Rx,-1,ERROR FRAME,3123792,True
3123792,1431.490477,1,ErrorFrame,Rx,-1,ERROR FRAME,3123793,True
3123793,1431.490501,1,ErrorFrame,Rx,-1,ERROR FRAME,3123794,True
3123794,1431.490517,1,ErrorFrame,Rx,-1,ERROR FRAME,3123795,True


In [9]:
# Error가 있는 행은 drop
df1 = df1[df1["Error"] != True]
df1 = df1.drop("Error", axis=1)

# df2 = df2[df2["Error"] != True]
# df2 = df2.drop("Error", axis=1)

# df3 = df3[df3["Error"] != True]
# df3 = df3.drop("Error", axis=1)

# df4 = df4[df4["Error"] != True]
# df4 = df4.drop("Error", axis=1)

# df5 = df5[df5["Error"] != True]
# df5 = df5.drop("Error", axis=1)

In [10]:
df1

,Time,CAN,Identifier,Flags,DLC,Data,Counter
0,0.026248,1,902,Rx,8,b'n\x84e\x04m\xc4c\xc4',1
1,0.026416,1,356,Rx,4,b'\x00\x08\x0c\x04',2
2,0.028296,1,1427,Rx,6,"b'\x00\x01!""""!'",3
3,0.028528,1,899,Rx,8,b'\x05(\x94qq\x00\x00\xb9',4
4,0.029188,1,897,Rx,8,b'\x80\x00@\x80\xffE\x85\x01',5
...,...,...,...,...,...,...,...
3123782,1431.486713,1,356,Rx,4,b'\x00\x08\x18\x10',3123783
3123783,1431.487789,1,273,Rx,8,b'\x00\xc5y\xff\x00\x00\x00\xc5',3123784
3123784,1431.488031,1,274,Rx,8,b'\xfe\x91\x00\x00g\x00\x00\x00',3123785
3123785,1431.488271,1,275,Rx,8,b'\x00\x9f\x10\x80\x00\x00\x00\xb4',3123786


In [11]:
# 이미 로드된 모듈 다시 로드
import importlib
from data_loader import DataLoader
importlib.reload(data_loader)

# DataLoader 인스턴스 생성
loader = DataLoader()

# 데이터프레임 저장
loader.save_dataframe(df1, 'df1')

NameError: name 'data_loader' is not defined

# deserialize 시작

In [ ]:
import pandas as pd
from data_loader import DataLoader

# DataLoader 인스턴스 생성
loader = DataLoader()

# 특정 캐시 파일 직접 읽기
from pathlib import Path
cache_path = Path('./cache/df1.parquet')

# 파일 존재 여부 확인
if cache_path.exists():
    # 파일이 있다면 직접 pandas로 읽기
    df1 = pd.read_parquet(cache_path)
    print(f"파일 로드 완료: {df1.shape}")
else:
    print("파일이 존재하지 않습니다.")

파일 로드 완료: (3123785, 7)


In [ ]:
Identifiers = df1["Identifier"].unique().tolist()
print(Identifiers)

['902', '356', '1427', '899', '897', '593', '688', '790', '608', '128', '129', '273', '274', '275', '339', '354', '399', '512', '544', '809', '1151', '1168', '1282', '1287', '903', '1078', '1265', '1292', '1312', '1331', '1332', '1333', '1345', '1170', '1351', '1353', '1366', '1367', '1349', '1419', '1365', '1280', '1530', '1356', '1040', '1322', '1314', '1348', '1363', '1415', '1369', '1456', '1460', '1470', '1491', '127', '1472', '66', '67', '68', '1440', '1407']


In [ ]:
import cantools

data = df1.loc[0,'Data']

db_2015_ccan = cantools.database.load_file('datasets/hyundai_2015_ccan.dbc')
decoded = pd.Series(db_2015_ccan.decode_message(902, data))

print('*** Deserialized sensor values (902) ***\n', decoded)

*** Deserialized sensor values (902) ***
 WHL_SPD_FL                  35.43750
WHL_SPD_AliveCounter_LSB     2.00000
WHL_SPD_FR                  35.15625
WHL_SPD_AliveCounter_MSB     0.00000
WHL_SPD_RL                  35.40625
WHL_SPD_Checksum_LSB         3.00000
WHL_SPD_RR                  35.09375
WHL_SPD_Checksum_MSB         3.00000
dtype: float64


In [ ]:
df1_serialized_902 = df1[df1["Identifier"]=='902']
df1_serialized_902

,Time,CAN,Identifier,Flags,DLC,Data,Counter
0,0.026248,1,902,Rx,8,b'n\x84e\x04m\xc4c\xc4',1
49,0.046237,1,902,Rx,8,b'q\xc4^\x04hDc\xc4',50
94,0.066232,1,902,Rx,8,b'o\x04gDi\xc4a\xc4',95
135,0.086229,1,902,Rx,8,b'nDeDj\xc4]\xc4',136
175,0.106233,1,902,Rx,8,b'm\x84`Dd\x84^\x84',176
...,...,...,...,...,...,...,...
3123607,1431.406569,1,902,Rx,8,b'\x01@\x01\x80\x01@\x01\xc0',3123608
3123653,1431.426564,1,902,Rx,8,b'\x01\x80\x01\x80\x01@\x01\xc0',3123654
3123693,1431.446555,1,902,Rx,8,b'\x01\xc0\x01\x80\x01@\x01\xc0',3123694
3123736,1431.466541,1,902,Rx,8,b'\x01\x00\x01\xc0\x01@\x01\xc0',3123737


In [ ]:
(db_2015_ccan.decode_message(902, data))

{'WHL_SPD_FL': 35.4375,
 'WHL_SPD_AliveCounter_LSB': 2,
 'WHL_SPD_FR': 35.15625,
 'WHL_SPD_AliveCounter_MSB': 0,
 'WHL_SPD_RL': 35.40625,
 'WHL_SPD_Checksum_LSB': 3,
 'WHL_SPD_RR': 35.09375,
 'WHL_SPD_Checksum_MSB': 3}

In [ ]:
type(df1_serialized_902.loc[0,"Identifier"])
df1_serialized_902 = df1[df1["Identifier"]=='902']

In [ ]:
def parsing_identifiers(dbc_file_path):
    """
    dbc_file_path: can bus 정보가 있는 파일의 경로 ('datasets/hyundai_2015_ccan.dbc')
    
    Return

    {
        'ID': {
            'name': '메시지 이름',
            'signals': ['시그널1', '시그널2', '시그널3', ...]
        },
        ...
    }

    """
    result = {}
    
    with open(dbc_file_path, 'r') as file:
        lines = file.readlines()
        
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        
        if line.startswith('BO_'):
            # BO_ 라인 파싱
            parts = line.split()
            if len(parts) >= 3:  # BO_ 1412 AAF11: 8 AAF 형식 확인
                identifier = int(parts[1])  # CAN ID (예: 1412)
                message_name = parts[2].rstrip(':')  # 메시지 이름 (예: AAF11)
                
                # 결과 딕셔너리에 메시지 ID를 키로 하고 시그널 목록을 저장할 리스트 초기화
                result[identifier] = {"name": message_name, "signals": []}
                
                # 다음 라인부터 순회하면서 SG_ 시그널 정보 수집
                j = i + 1
                while j < len(lines) and lines[j].strip().startswith('SG_'):
                    signal_line = lines[j].strip()
                    signal_parts = signal_line.split()
                    if len(signal_parts) >= 2:
                        signal_name = signal_parts[1]  # 시그널 이름
                        result[identifier]["signals"].append(signal_name)
                    j += 1
                
                # 다음 BO_ 찾기 위해 인덱스 업데이트
                i = j - 1  # 다음 반복에서 i += 1이 실행되므로 j-1로 설정
        
        i += 1
        
    return result

# 사용 예
parsed_ID = (parsing_identifiers('datasets/hyundai_2015_ccan.dbc'))
print(parsed_ID)

{1532: {'name': 'ODS13', 'signals': ['CR_Ods_ID', 'CR_Ods_Chksum_H', 'CR_Ods_Chksum_L', 'CR_Ods_RomID_H', 'CR_Ods_RomID_L']}, 1531: {'name': 'ODS12', 'signals': ['CR_Ods_SerNum0', 'CR_Ods_SerNum1', 'CR_Ods_SerNum2', 'CR_Ods_SerNum3', 'CR_Ods_SerNum4', 'CR_Ods_SerNum5', 'CR_Ods_SerNum6', 'CR_Ods_SerNum7']}, 1530: {'name': 'ODS11', 'signals': ['CF_Ods_PrcCmd', 'CF_Ods_BtsFail', 'CF_Ods_AcuRcvSN', 'CF_Ods_EolCal', 'CF_Ods_PsFail', 'CF_Ods_EcuFail', 'CF_Ods_WgtStat', 'CF_Ods_OccStat', 'CR_Wcs_ErrStat', 'CR_Wcs_ClassStat']}, 1017: {'name': 'ECS12', 'signals': ['Height_FL', 'Height_FR', 'Height_RL', 'Height_RR']}, 1268: {'name': 'SPAS12', 'signals': ['CF_Spas_HMI_Stat', 'CF_Spas_Disp', 'CF_Spas_FIL_Ind', 'CF_Spas_FIR_Ind', 'CF_Spas_FOL_Ind', 'CF_Spas_FOR_Ind', 'CF_Spas_VolDown', 'CF_Spas_RIL_Ind', 'CF_Spas_RIR_Ind', 'CF_Spas_FLS_Alarm', 'CF_Spas_ROL_Ind', 'CF_Spas_ROR_Ind', 'CF_Spas_FCS_Alarm', 'CF_Spas_FI_Ind', 'CF_Spas_RI_Ind', 'CF_Spas_FRS_Alarm', 'CF_Spas_FR_Alarm', 'CF_Spas_RR_Alarm', '

In [ ]:
def deserialize(identifier, df,time=False):
    """
    identifier => 정수 입력
    df => Time	CAN	Identifier	Flags	DLC	Data	Counter 컬럼들이 있는 데이터프레임
    """
    filtered_df = df[df["Identifier"]==str(identifier)]

    # Data컬럼 solution1: apply 메서드
    filtered_df = filtered_df.copy()
    filtered_df['Decoded'] = filtered_df['Data'].apply(lambda x: db_2015_ccan.decode_message(identifier, x))
    # 결과를 데이터프레임으로 변환
    # decoded_series에서 딕셔너리 목록 추출
    decoded_dicts = filtered_df['Decoded'].tolist()
    
    # 딕셔너리 목록을 데이터프레임으로 변환
    result = pd.DataFrame(decoded_dicts, index=filtered_df.index)

    # 원본 데이터프레임의 Time 컬럼 추가
    result['Time'] = filtered_df['Time']

    
    # Time 컬럼을 datetime 형식으로 변환
    result['Time'] = pd.to_timedelta(result['Time'], unit='s')  # 초 단위로 해석
    
    # Time을 인덱스로 설정
    result = result.set_index('Time')

    print(f"time duplicated 처리 전 데이터프레임의 길이: {len(result)}")

    # 파라미터로 입력한 시간을 기준으로 반올림 후 중복된 시간 인덱스 중 가장 마지막 인덱스만 살리는 작업
    if time==False:
        # time값이 입력되지 않았으므로 모든 행을 그대로 둠
        pass
    else:
        # time값이 입력되었으므로 작업을 수행함
        result.index = result.index.ceil(time)
        # print(result.to_string())
        result = result[~result.index.duplicated(keep='last')]
        # print(f"time duplicated 처리 후 데이터프레임의 길이: {len(result)}")
    
    # TODO min ~ Max 사이 t 간격으로 모든 index

    if time:
        index_new = pd.timedelta_range(start=result.index.min(),
                                       end=result.index.max(), freq=time)
        result = result.reindex(index=index_new)

    # 바꿀 때 arbitrationID_컬럼명 이렇게 바꿔준다 (컬럼명 중복이 가끔 있음)
    
    identifier = hex(int(identifier))[2:]  # 16진수 변환. 인덱싱은 앞 0x를 제거하기 위함
    result.columns = [f"{identifier}_{col}" for col in result.columns]

    return result



In [ ]:
df_deserialized_902 = deserialize(902, df1,'0.25s')
# print(df_deserialized_902.to_string())
df_deserialized_902

time duplicated 처리 전 데이터프레임의 길이: 71580


,902_WHL_SPD_FL,902_WHL_SPD_AliveCounter_LSB,902_WHL_SPD_FR,902_WHL_SPD_AliveCounter_MSB,902_WHL_SPD_RL,902_WHL_SPD_Checksum_LSB,902_WHL_SPD_RR,902_WHL_SPD_Checksum_MSB
0 days 00:00:00.250000,35.15625,1,34.84375,3,35.25000,1,34.87500,3
0 days 00:00:00.500000,34.96875,1,34.56250,2,34.81250,2,34.50000,2
0 days 00:00:00.750000,34.90625,2,34.65625,1,34.00000,0,34.09375,2
0 days 00:00:01,34.43750,2,34.43750,0,34.31250,2,34.59375,2
0 days 00:00:01.250000,34.53125,3,34.37500,3,34.31250,1,34.18750,2
...,...,...,...,...,...,...,...,...
0 days 00:23:50.500000,0.06250,3,0.06250,2,0.06250,1,0.06250,3
0 days 00:23:50.750000,0.03125,0,0.06250,2,0.03125,1,0.06250,3
0 days 00:23:51,0.03125,0,0.03125,1,0.03125,1,0.03125,3
0 days 00:23:51.250000,0.03125,1,0.03125,0,0.03125,1,0.03125,3


In [ ]:
try:
    df_deserialized_902 = deserialize(902, df1) # 세번째 파라미터는 시간 형식으로 입력하거나, 아예 입력하지 않아야 한다.
    df_deserialized_356 = deserialize(356, df1)
    df_merged = pd.concat([df_deserialized_902, df_deserialized_356], axis=1)
    df_merged = df_merged.sort_index()
except Exception as e:
    print(f"다음 에러가 발생: {e}\n 0.25s값을 time 파라미터로 입력하여 다시 시도합니다.")
    df_deserialized_902 = deserialize(902, df1,'0.25s') # 세번째 파라미터는 시간 형식으로 입력하거나, 아예 입력하지 않아야 한다.
    df_deserialized_356 = deserialize(356, df1,'0.25s')
    df_merged = pd.concat([df_deserialized_902, df_deserialized_356], axis=1)
    df_merged = df_merged.sort_index()
df_merged

time duplicated 처리 전 데이터프레임의 길이: 71580
time duplicated 처리 전 데이터프레임의 길이: 143159


,WHL_SPD_FL,WHL_SPD_AliveCounter_LSB,WHL_SPD_FR,WHL_SPD_AliveCounter_MSB,WHL_SPD_RL,WHL_SPD_Checksum_LSB,WHL_SPD_RR,WHL_SPD_Checksum_MSB,CR_Esc_StrTqReq,CF_Esc_Act,CF_Esc_CtrMode,CF_Esc_Def,CF_Esc_AliveCnt,CF_Esc_Chksum
Time,,,,,,,,,,,,,,
0 days 00:00:00.026248,35.43750,2.0,35.15625,0.0,35.40625,3.0,35.09375,3.0,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.026416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,6.0,4.0
0 days 00:00:00.036303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,7.0,6.0
0 days 00:00:00.046237,35.53125,3.0,34.93750,0.0,35.25000,1.0,35.09375,3.0,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.046405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,8.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 00:23:51.466541,0.03125,0.0,0.03125,3.0,0.03125,1.0,0.03125,3.0,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:23:51.466710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,10.0,28.0
0 days 00:23:51.476603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,11.0,30.0


In [ ]:
A = set(df_deserialized_902.index)
B = set(df_deserialized_356.index)

A-B


{Timedelta('0 days 00:12:12.264735'),
 Timedelta('0 days 00:17:20.938831'),
 Timedelta('0 days 00:00:31.843569'),
 Timedelta('0 days 00:01:25.979026'),
 Timedelta('0 days 00:00:46.443227'),
 Timedelta('0 days 00:09:36.918725'),
 Timedelta('0 days 00:05:28.918623'),
 Timedelta('0 days 00:01:06.440661'),
 Timedelta('0 days 00:15:21.728828'),
 Timedelta('0 days 00:14:09.354905'),
 Timedelta('0 days 00:19:45.146987'),
 Timedelta('0 days 00:06:52.314745'),
 Timedelta('0 days 00:10:56.431078'),
 Timedelta('0 days 00:05:46.957109'),
 Timedelta('0 days 00:05:13.520324'),
 Timedelta('0 days 00:02:41.852660'),
 Timedelta('0 days 00:17:27.178544'),
 Timedelta('0 days 00:17:17.999071'),
 Timedelta('0 days 00:19:57.445954'),
 Timedelta('0 days 00:22:07.777222'),
 Timedelta('0 days 00:00:25.124145'),
 Timedelta('0 days 00:04:32.383382'),
 Timedelta('0 days 00:19:12.049533'),
 Timedelta('0 days 00:01:07.900539'),
 Timedelta('0 days 00:05:54.116507'),
 Timedelta('0 days 00:12:42.782139'),
 Timedelta('

# Merge

In [ ]:
print(parsed_ID)

{1532: {'name': 'ODS13', 'signals': ['CR_Ods_ID', 'CR_Ods_Chksum_H', 'CR_Ods_Chksum_L', 'CR_Ods_RomID_H', 'CR_Ods_RomID_L']}, 1531: {'name': 'ODS12', 'signals': ['CR_Ods_SerNum0', 'CR_Ods_SerNum1', 'CR_Ods_SerNum2', 'CR_Ods_SerNum3', 'CR_Ods_SerNum4', 'CR_Ods_SerNum5', 'CR_Ods_SerNum6', 'CR_Ods_SerNum7']}, 1530: {'name': 'ODS11', 'signals': ['CF_Ods_PrcCmd', 'CF_Ods_BtsFail', 'CF_Ods_AcuRcvSN', 'CF_Ods_EolCal', 'CF_Ods_PsFail', 'CF_Ods_EcuFail', 'CF_Ods_WgtStat', 'CF_Ods_OccStat', 'CR_Wcs_ErrStat', 'CR_Wcs_ClassStat']}, 1017: {'name': 'ECS12', 'signals': ['Height_FL', 'Height_FR', 'Height_RL', 'Height_RR']}, 1268: {'name': 'SPAS12', 'signals': ['CF_Spas_HMI_Stat', 'CF_Spas_Disp', 'CF_Spas_FIL_Ind', 'CF_Spas_FIR_Ind', 'CF_Spas_FOL_Ind', 'CF_Spas_FOR_Ind', 'CF_Spas_VolDown', 'CF_Spas_RIL_Ind', 'CF_Spas_RIR_Ind', 'CF_Spas_FLS_Alarm', 'CF_Spas_ROL_Ind', 'CF_Spas_ROR_Ind', 'CF_Spas_FCS_Alarm', 'CF_Spas_FI_Ind', 'CF_Spas_RI_Ind', 'CF_Spas_FRS_Alarm', 'CF_Spas_FR_Alarm', 'CF_Spas_RR_Alarm', '

In [ ]:
print(Identifiers)

['902', '356', '1427', '899', '897', '593', '688', '790', '608', '128', '129', '273', '274', '275', '339', '354', '399', '512', '544', '809', '1151', '1168', '1282', '1287', '903', '1078', '1265', '1292', '1312', '1331', '1332', '1333', '1345', '1170', '1351', '1353', '1366', '1367', '1349', '1419', '1365', '1280', '1530', '1356', '1040', '1322', '1314', '1348', '1363', '1415', '1369', '1456', '1460', '1470', '1491', '127', '1472', '66', '67', '68', '1440', '1407']


In [ ]:
# 빈 리스트 생성
store =[]
t = '0.0025s'

# 각 그룹(unique한 Identifier)에 대해 for문 실행
for identifier in Identifiers:
    print(f"\n\n{(Identifiers.index(identifier)+1)}번 째 / 전체 {len(Identifiers)} 번")
    identifier = int(identifier)
    print(f"현재 처리 중인 Identifier: {identifier}")
    if identifier in parsed_ID:

        print(f"{identifier}가 CAN.dbc 파일에 존재합니다. 인덱스는 Time이며, Deserialize 작업을 시작합니다.")
        
        # deserialize 함수 호출하여 데이터 변환
        new_df = deserialize(identifier, df1, t)
        store.append(new_df)
        
    else:
        print(f"{identifier}(이)가 list에 없거나 무언가 잘못되었습니다.")

# concat은 루프 밖에서하고, concat한 데이터프레임 출력
df_merged = pd.concat(store, axis=1, verify_integrity=False)
df_merged = df_merged[~df_merged.index.duplicated(keep='last')]
df_merged




1번 째 / 전체 62 번
현재 처리 중인 Identifier: 902
902가 CAN.dbc 파일에 존재합니다. 인덱스는 Time이며, Deserialize 작업을 시작합니다.
time duplicated 처리 전 데이터프레임의 길이: 71580


2번 째 / 전체 62 번
현재 처리 중인 Identifier: 356
356가 CAN.dbc 파일에 존재합니다. 인덱스는 Time이며, Deserialize 작업을 시작합니다.
time duplicated 처리 전 데이터프레임의 길이: 143159


3번 째 / 전체 62 번
현재 처리 중인 Identifier: 1427
1427가 CAN.dbc 파일에 존재합니다. 인덱스는 Time이며, Deserialize 작업을 시작합니다.
time duplicated 처리 전 데이터프레임의 길이: 7158


4번 째 / 전체 62 번
현재 처리 중인 Identifier: 899
899가 CAN.dbc 파일에 존재합니다. 인덱스는 Time이며, Deserialize 작업을 시작합니다.
time duplicated 처리 전 데이터프레임의 길이: 71741


5번 째 / 전체 62 번
현재 처리 중인 Identifier: 897
897가 CAN.dbc 파일에 존재합니다. 인덱스는 Time이며, Deserialize 작업을 시작합니다.
time duplicated 처리 전 데이터프레임의 길이: 71571


6번 째 / 전체 62 번
현재 처리 중인 Identifier: 593
593가 CAN.dbc 파일에 존재합니다. 인덱스는 Time이며, Deserialize 작업을 시작합니다.
time duplicated 처리 전 데이터프레임의 길이: 143140


7번 째 / 전체 62 번
현재 처리 중인 Identifier: 688
688가 CAN.dbc 파일에 존재합니다. 인덱스는 Time이며, Deserialize 작업을 시작합니다.
time duplicated 처리 전 데이터프레임의 길이: 143140


8번 째 / 

,386_WHL_SPD_FL,386_WHL_SPD_AliveCounter_LSB,386_WHL_SPD_FR,386_WHL_SPD_AliveCounter_MSB,386_WHL_SPD_RL,386_WHL_SPD_Checksum_LSB,386_WHL_SPD_RR,386_WHL_SPD_Checksum_MSB,164_CR_Esc_StrTqReq,164_CF_Esc_Act,...,5a0_CF_PasBkl_FltStat,5a0_CF_DriBkl_FltStat,5a0_CF_PasBkl_Stat,5a0_CF_DriBkl_Stat,5a0_CF_SWL_Ind,5a0_CF_Acu_FltStat,5a0_CF_Acu_ExtOfSab,5a0_CF_Acu_Dtc,5a0_CF_Acu_NumOfFlt,57f_HU_Type
0 days 00:00:00.027500,35.4375,2.0,35.15625,0.0,35.40625,3.0,35.09375,3.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.032500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.035000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.037500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 00:23:51.480000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:23:51.482500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:23:51.485000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:23:51.487500,0.0000,1.0,0.03125,3.0,0.03125,2.0,0.03125,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
importlib.reload(data_loader)
name = 'df_merged' + '_' + t[2:]
loader.save_dataframe(df_merged, name, write=True)

df_merged_0025s.parquet 파일이 이미 존재하지만 덮어씌웁니다.


True

In [ ]:
# 시간간격 조절하여 데이터 로드 가능
t = '0.0025s'

name = './cache/df_merged_' + t[2:] + '.parquet'
df_merged = pd.read_parquet(name)
df_merged

,386_WHL_SPD_FL,386_WHL_SPD_AliveCounter_LSB,386_WHL_SPD_FR,386_WHL_SPD_AliveCounter_MSB,386_WHL_SPD_RL,386_WHL_SPD_Checksum_LSB,386_WHL_SPD_RR,386_WHL_SPD_Checksum_MSB,164_CR_Esc_StrTqReq,164_CF_Esc_Act,...,5a0_CF_PasBkl_FltStat,5a0_CF_DriBkl_FltStat,5a0_CF_PasBkl_Stat,5a0_CF_DriBkl_Stat,5a0_CF_SWL_Ind,5a0_CF_Acu_FltStat,5a0_CF_Acu_ExtOfSab,5a0_CF_Acu_Dtc,5a0_CF_Acu_NumOfFlt,57f_HU_Type
0 days 00:00:00.027500,35.4375,2.0,35.15625,0.0,35.40625,3.0,35.09375,3.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.032500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.035000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.037500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 00:23:51.480000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:23:51.482500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:23:51.485000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:23:51.487500,0.0000,1.0,0.03125,3.0,0.03125,2.0,0.03125,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
the_107_signals = [
    '044_CR_Datc_OutTempC', '044_CF_Datc_IncarTemp', '080_PV_AV_CAN', '080_N', '080_TQI_ACOR',
    '080_TQFR', '080_TQI', '081_BRAKE_ACT', '081_CF_Ems_EngOperStat', '081_CR_Ems_IndAirTemp',
    '111_TQI_TCU_INC', '111_SWI_GS', '111_TQI_TCU', '111_SWI_CC', '112_VS_TCU', '112_N_INC_TCU',
    '112_VS_TCU_DECIMAL', '113_SLOPE_TCU', '113_CF_Tcu_TarGr', '113_CF_Tcu_ShfPatt', '113_CF_Tcu_ActEcoRdy',
    '162_Clutch_Driving_Tq', '162_Cluster_Engine_RPM', '162_Cluster_Engine_RPM_Flag', '18F_R_PAcnC', '18F_TQI_B',
    '18F_R_NEngIdlTgC', '200_FCO', '220_LAT_ACCEL', '220_LONG_ACCEL', '220_CYL_PRES', '220_YAW_RATE',
    '251_CR_Mdps_StrTq', '251_CR_Mdps_OutTq', '260_TQI_MIN', '260_TQI', '260_TQI_TARGET', '260_TQI_MAX',
    '260_CF_Ems_AclAct', '2B0_SAS_Angle', '2B0_SAS_Speed', '316_PUC_STAT', '316_TQI_ACOR', '316_N', '316_TQI',
    '316_TQFR', '316_VS', '329_TEMP_ENG', '329_MAF_FAC_ALTI_MMV', '329_CLU_ACK', '329_BRAKE_ACT', '329_TPS',
    '329_PV_AV_CAN', '381_CR_Mdps_StrAng', '383_CR_Fatc_OutTemp', '383_CR_Fatc_OutTempSns', '386_WHL_SPD_FL',
    '386_WHL_SPD_FR', '386_WHL_SPD_RL', '386_WHL_SPD_RR', '387_WHL_PUL_FL', '387_WHL_PUL_FR', '387_WHL_PUL_RL',
    '387_WHL_PUL_RR', '47F_ROL_CNT_ESP', '4F1_CF_Clu_VanzDecimal', '4F1_CF_Clu_Vanz', '4F1_CF_Clu_DetentOut',
    '50C_CF_Clu_AvgFCI', '50C_CF_Clu_DTE', '52A_CF_Clu_VehicleSpeed', '541_CF_Gway_TurnSigLh',
    '541_CF_Gway_HeadLampLow', '541_CF_Gway_TurnSigRh', '545_AMP_CAN', '545_BAT_Alt_FR_Duty',
    '545_VB', '545_TEMP_FUEL', '547_ECGPOvrd', '547_IntAirTemp', '549_BAT_SNSR_I', '549_BAT_SOC',
    '549_BAT_SNSR_V', '549_BAT_SNSR_Temp', '553_CF_Gway_AutoLightValue', '553_CF_Gway_AvTail',
    '553_CF_Gway_ExtTailAct', '553_CF_Gway_IntTailAct', '555_CR_Fpcm_LPActPre', '556_PID_04h',
    '556_PID_05h', '556_PID_0Ch', '556_PID_0Dh', '556_PID_11h', '557_PID_0Bh', '557_PID_23h',
    '58B_CF_Lca_Stat', '58B_CF_Lca_IndLeft', '58B_CF_Lca_IndRight', '58B_CF_Lca_IndBriLeft',
    '58B_CF_Lca_IndBriRight', '593_PRESSURE_FL', '593_PRESSURE_FR', '593_PRESSURE_RL',
    '593_PRESSURE_RR', '5A0_CF_Acu_Dtc', '5B0_CF_Clu_Odometer'
]

for i in my:
    if i in the_107_signals:
        print(i)

# df_merged = df_merged[[col for col in the_107_signals if col in df_merged.columns]]


In [ ]:
df_merged.head(30)

,386_WHL_SPD_FL,386_WHL_SPD_AliveCounter_LSB,386_WHL_SPD_FR,386_WHL_SPD_AliveCounter_MSB,386_WHL_SPD_RL,386_WHL_SPD_Checksum_LSB,386_WHL_SPD_RR,386_WHL_SPD_Checksum_MSB,164_CR_Esc_StrTqReq,164_CF_Esc_Act,...,5a0_CF_PasBkl_FltStat,5a0_CF_DriBkl_FltStat,5a0_CF_PasBkl_Stat,5a0_CF_DriBkl_Stat,5a0_CF_SWL_Ind,5a0_CF_Acu_FltStat,5a0_CF_Acu_ExtOfSab,5a0_CF_Acu_Dtc,5a0_CF_Acu_NumOfFlt,57f_HU_Type
0 days 00:00:00.027500,35.43750,2.0,35.15625,0.0,35.40625,3.0,35.09375,3.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.032500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.035000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.037500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.040000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.042500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.045000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.047500,35.53125,3.0,34.93750,0.0,35.25000,1.0,35.09375,3.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.050000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_merged = df_merged.ffill()
df_merged

,386_WHL_SPD_FL,386_WHL_SPD_AliveCounter_LSB,386_WHL_SPD_FR,386_WHL_SPD_AliveCounter_MSB,386_WHL_SPD_RL,386_WHL_SPD_Checksum_LSB,386_WHL_SPD_RR,386_WHL_SPD_Checksum_MSB,164_CR_Esc_StrTqReq,164_CF_Esc_Act,...,5a0_CF_PasBkl_FltStat,5a0_CF_DriBkl_FltStat,5a0_CF_PasBkl_Stat,5a0_CF_DriBkl_Stat,5a0_CF_SWL_Ind,5a0_CF_Acu_FltStat,5a0_CF_Acu_ExtOfSab,5a0_CF_Acu_Dtc,5a0_CF_Acu_NumOfFlt,57f_HU_Type
0 days 00:00:00.027500,35.43750,2.0,35.15625,0.0,35.40625,3.0,35.09375,3.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.030000,35.43750,2.0,35.15625,0.0,35.40625,3.0,35.09375,3.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.032500,35.43750,2.0,35.15625,0.0,35.40625,3.0,35.09375,3.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.035000,35.43750,2.0,35.15625,0.0,35.40625,3.0,35.09375,3.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0 days 00:00:00.037500,35.43750,2.0,35.15625,0.0,35.40625,3.0,35.09375,3.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0 days 00:23:51.480000,0.03125,0.0,0.03125,3.0,0.03125,1.0,0.03125,3.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,2.0,38035.0,2.0,96.0
0 days 00:23:51.482500,0.03125,0.0,0.03125,3.0,0.03125,1.0,0.03125,3.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,2.0,38035.0,2.0,96.0
0 days 00:23:51.485000,0.03125,0.0,0.03125,3.0,0.03125,1.0,0.03125,3.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,2.0,38035.0,2.0,96.0
0 days 00:23:51.487500,0.00000,1.0,0.03125,3.0,0.03125,2.0,0.03125,2.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,2.0,38035.0,2.0,96.0
